# HMGP Finance Merge
The goal of this notebook is to merge the HMGP_ACS data with city finance data by city, county fips, and year. It might be fuzzy due to issues with city names, but the goal should be to merge around 2/3 of the available data.

In [3]:
# importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
import scipy.stats as stats


## Downloading and Inspecting HMGP and Finance Data
The next few cells will read and inspect the files to make sure they are mergeable.

In [4]:
# reading the separate files

# HMGP_ACS file
HMGP_ACS = pd.read_csv(r'HMGP_ACS.csv')

# city finance file
finance = pd.read_csv(r'city finance.csv')


In [7]:
# Checking pure city matches (not considering countyfips or year)

print(f'Num matches in HMGP_ACS: {len(HMGP_ACS.loc[HMGP_ACS["city"].isin(finance.city.values),:])}')
print(f'Num matches in finance: {len(finance.loc[finance["city"].isin(HMGP_ACS.city.values),:])}')


Num matches in HMGP_ACS: 499607
Num matches in finance: 355380


In [ ]:
# percentage of cities from city finance dataset that match with HMGP_ACS

print(f'Pct:',len(finance.loc[(finance["year"].isin(HMGP_ACS.year.values))\
     &(finance["countyfips"].isin(HMGP_ACS.countyfips.values))&\
     (finance["city"].isin(HMGP_ACS.city.values)),:])/len(finance))


In [ ]:
# 1 sample t-test of percentage of cities in each state

# total states in city finance dataset
print(f'# of states: {len(np.unique(finance.statefips.values))}\n')

# creating the percentages for each state
pct_states = {}
for state in np.unique(finance.statefips.values):
    pct_states[state] = len(finance.loc[(~finance["city"].isin(HMGP_ACS.\
                       city.values))&(finance["statefips"] == state),:])\
                       /len(finance.loc[(~finance["city"].isin(HMGP_ACS.\
                       city.values)),:])
print(f'Mean pct: {np.mean(list(pct_states.values()))*100}\n'
      f'Predicted pct: {(1/51)*100}\n')

# running t-test
result = stats.ttest_1samp(list(pct_states.values()), 1/51)
print(f'P-value: {result.pvalue}, therefore no statistical diff.\n')

# state with most unmatched cities + pct
largest_pct = np.max(list(pct_states.values()))
largest_st = list(pct_states.keys())[list(pct_states.values()).index(\
                                    largest_pct)]
print(f'State with largest pct & pct: {largest_st}, {largest_pct}\n')

# st fips of texas: 48, st fips of california: 06
print(finance.loc[(~finance["city"].isin(HMGP_ACS.\
                       city.values))&(finance["statefips"] == 48),:])

# dict of states and pcts sorted from small to large
sorted_stpct = dict(sorted(pct_states.items(), key=lambda item: item[1]))


## Merging the Datasets
The next few cells will merge the datasets by adding the new city finance data to the already matching cities and creating new rows for the mis-matched cities with only their city finance data.

In [ ]:
# "nice" merge

# defining the columns of interest for "nice" merge
finance_cols = ["city_population","city_revpc","city_ownrev","city_tax",\
                "city_igrev","city_spend"]
HMGP_ACS_CF = HMGP_ACS.reindex(columns = \
                               HMGP_ACS.columns.tolist() + finance_cols)
HMGP_ACS_CF.sort_values(["city", "countyfips", "year"],\
                        inplace=True, ignore_index=True)

# merging by city and county
nice_finance = finance.loc[(finance["year"].isin(HMGP_ACS.year.values))\
     &(finance["countyfips"].isin(HMGP_ACS.countyfips.values))&\
     (finance["city"].isin(HMGP_ACS.city.values)),:]

for city in tqdm(np.unique(nice_finance.city.values), \
                 total = len(np.unique(nice_finance.city.values))):
    for fips in np.unique(nice_finance[nice_finance["city"] == city].\
                          countyfips.values):
        finance = (nice_finance.loc[(nice_finance["city"] == city)&\
                  (nice_finance["countyfips"] == fips)]).\
                   iloc[:,-len(finance_cols):]
        nan_mat = np.empty(2019-2000)
        nan_mat[:] = np.NaN
        for i, col in enumerate(finance_cols):
            HMGP_ACS_CF.loc[(HMGP_ACS_CF["city"] == city)&\
            (HMGP_ACS_CF["countyfips"] == fips),col] \
            = HMGP_ACS_CF.loc[(HMGP_ACS_CF["city"] == city)&\
            (HMGP_ACS_CF["countyfips"] == fips),col].\
            replace([nan_mat],[finance.loc[:,col].values])


In [ ]:
# Saving file with "nice" merge

# reordering rows
HMGP_ACS_CF.sort_values(["city", "countyfips", "year"], \
                        inplace=True, ignore_index=True)
HMGP_ACS_CF.to_csv('HMGP_ACS_CF_nice1.csv')


In [215]:
# "not nice" merge (aka non-matching cities)

# reading file
HMGP_ACS_CF_final = pd.read_csv('HMGP_ACS_CF_nice.csv')

# getting rid of unwanted HMGP columns
HMGP_ACS_CF_final.drop(["Unnamed: 0.1", "Unnamed: 0", \
                        "fips", "stfips"], inplace = True, axis = 1)

# values we'd like to add to HMGP_ACS_CF 
nn_finance = finance.loc[~(finance["countyfips"].isin(HMGP_ACS.\
    countyfips.values))|~(finance["city"].isin(HMGP_ACS.city.values)),:]


In [216]:
# fixing HMGP_ACS_CF and nn_finance columns to best fit not nice data

# reordering HMGP columns
cols = HMGP_ACS_CF_final.columns.tolist()
cols = cols[:-20] + cols[-18:-6] + [cols[-20]] + [cols[-19]] + cols[-6:]
cols = cols[:2] + cols[5:-6] + [cols[3]] + [cols[4]] + [cols[2]]\
                                                      + cols[-6:]
HMGP_ACS_CF_final = HMGP_ACS_CF_final[cols]

# Adding nan columns to nn_finance
nan_mat = np.empty(len(nn_finance))
nan_mat[:] = np.NaN
for i, col in enumerate(HMGP_ACS_CF_final.columns.tolist()[:-10]):
    nn_finance.insert(i,col,nan_mat, True)


In [217]:
# combining both datasets to create final dataset, then reordering

# combining datasets with concat
HMGP_ACS_CF_final = pd.concat([HMGP_ACS_CF_final, nn_finance],\
                              ignore_index = True)

# dropping extra ZCTA5 columns
HMGP_ACS_CF_final.drop("ZCTA5", inplace = True, axis = 1)

# reordering
cols = HMGP_ACS_CF_final.columns.tolist()
cols = [cols[0]] + cols[-11:-6] + cols[1:-11] + cols[-6:]
HMGP_ACS_CF_final = HMGP_ACS_CF_final[cols]


In [218]:
# Saving file with both "nice" and "not nice" merge

# reordering rows
HMGP_ACS_CF_final.sort_values(["city", "countyfips", "year"], \
                        inplace=True, ignore_index=True)
HMGP_ACS_CF_final.to_csv('HMGP_ACS_CF.csv')
